- 1st lgb

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold

In [2]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")

# preprocess

In [3]:
train["imp_at"] = pd.to_datetime(train["imp_at"])

In [4]:
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)

In [5]:
train.drop(['request_id'], axis=1, inplace=True)
test.drop(['request_id'], axis=1, inplace=True)

In [6]:
train.columns

Index(['imp_at', 'target', 'adnw_id', 'adspot_id', 'adspot_video_format_id',
       'advertiser_id', 'app_id', 'auction_type_id', 'campaign_id',
       'category_id', 'country_code', 'first_login_interval', 'frequency',
       'from_click', 'game_feed_asset_type_id', 'game_feed_id',
       'game_template_id', 'header_bidding', 'is_interstitial', 'item_id',
       'last_login_interval', 'login_frequency', 'max_login_interval',
       'media_app_id', 'os', 'os_version', 'pos', 'uid', 'user_type_id',
       'video_template_id'],
      dtype='object')

In [7]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]

In [8]:
for i in object_feats:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(pd.concat([train[i], test[i]])))
    train[i] = lbl.transform(list(train[i]))
    test[i] = lbl.transform(list(test[i]))

In [9]:
mask = train.imp_at > "2020-05-21"
valid = train.loc[mask].copy()
train = train.loc[~mask]

In [10]:
train.shape, valid.shape

((1568715, 30), (428880, 30))

In [11]:
remove_features = ["imp_at", 'uid']
    
#for i in X_train.columns:
#    if (X_train[i].std() == 0) and i not in remove_features:
#        remove_features.append(i)
    
train = train.drop(remove_features, axis=1)
valid = valid.drop(remove_features, axis=1)
test = test.drop(remove_features, axis=1)

In [12]:
train.shape, valid.shape

((1568715, 28), (428880, 28))

In [13]:
train.head()

,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,country_code,...,item_id,last_login_interval,login_frequency,max_login_interval,media_app_id,os,os_version,pos,user_type_id,video_template_id
0,0,4,1111,2.0,20,14871,1.0,403,8497,0,...,NaN,11320.0,10.0,2607.0,11754,1,86,0,1,5
1,0,28,1081,2.0,121,8300,2.0,91,0,0,...,NaN,12589.0,2.0,13540.0,8994,2,151,1,2,3
2,0,42,1123,3.0,5,12947,1.0,114,0,0,...,80525.0,10254.0,9.0,33725.0,346,1,54,0,1,9
3,0,8,1113,2.0,22,12378,1.0,75,0,0,...,NaN,39618.0,4.0,7377.0,6144,1,90,0,1,6
4,0,22,1061,NaN,135,195,4.0,331,0,0,...,NaN,35652.0,5.0,164759.0,2189,1,70,0,1,9


# modelling

In [14]:
lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt',
               'tree_learner': 'serial', 'learning_rate': 0.03, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

y_train = train.target.copy()
X_train = train.drop(['target'],axis=1).copy()
y_valid = valid.target.copy()
X_valid = valid.drop(['target'],axis=1).copy()
    
valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
real = np.array([])
features_list = [i for i in X_train.columns]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
       num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10, feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
feature_importance_df["Importance"] = model.feature_importance()
        
#feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

score = average_precision_score(y_valid, valid_predict)
print("average precision score = {}".format(score))
pred_value = model.predict(test, num_iteration = model.best_iteration)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['advertiser_id', 'app_id', 'campaign_id', 'country_code', 'game_feed_id', 'game_template_id', 'media_app_id', 'os_version', 'uid', 'video_template_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


TypeError: Wrong type(str) or unknown name(uid) in categorical_feature

# submission

In [15]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = pred_value
lgb_sub.to_csv("atmacup3_simple_lgb_sub"+str(score)[:-10]+".csv", index = False)

NameError: name 'pred_value' is not defined